In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/Colab Notebooks"

'Colab, Kaggle and OpenML.ipynb'   IMDB_TRYING_VW.ipynb
'Copy of IMDB_TRYING_VW.ipynb'	   Iris_Adaline.py
'DAT300 information.ipynb'	   kaggle.json
'Feature Engineering.ipynb'	   movie_data.csv
 gender_submission.csv		   pickle_sentiment-appy.ipynb
 gs_lr_tfidf.pickle		   plot_discretization.ipynb
 IMDB_revisited_appy.ipynb	  'System stuff.ipynb'
'IMDB sentiment - VW.ipynb'	  'Titanic Survival.py'


In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [0]:
import re
def preprocessor(text):
    # Regular expression for HTML tags
    text = re.sub('<[^>]*>', '', text)
    
    # Most typical emoticons (smileys)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    
    # Remove all non-word characters, convert to lower-case and add possible emoticons to the end.
    text = (re.sub('[\W]+', ' ', text.lower()) +
            ' '.join(emoticons).replace('-', ''))
    return text

In [0]:
# This takes a few seconds
df['review'] = df['review'].apply(preprocessor)

In [0]:
df['review'].values

array(['in 1974 the teenager martha moxley maggie grace moves to the high class area of belle haven greenwich connecticut on the mischief night eve of halloween she was murdered in the backyard of her house and her murder remained unsolved twenty two years later the writer mark fuhrman christopher meloni who is a former la detective that has fallen in disgrace for perjury in o j simpson trial and moved to idaho decides to investigate the case with his partner stephen weeks andrew mitchell with the purpose of writing a book the locals squirm and do not welcome them but with the support of the retired detective steve carroll robert forster that was in charge of the investigation in the 70 s they discover the criminal and a net of power and money to cover the murder murder in greenwich is a good tv movie with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a kennedy the powerful and rich family used their influence to cover 

In [0]:
import nltk

# Update to most resent stop-words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

# Define basic tokenizer and Porter stemmer version
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [0]:
from nltk.corpus import stopwords

# Combine tokenizer with Porter stemmer and stop-word removal
stop = stopwords.words('english')
#[w for w in tokenizer_porter(df['review'].values)
#if w not in stop]

#[w for w in tokenizer_porter('a runner likes running and runs a lot')
#if w not in stop]

In [0]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [0]:
stops = []
for s in stop:
    stops.append(tokenizer(s)[0])
stopsPorter = []
for s in stop:
    stopsPorter.append(tokenizer_porter(s)[0])

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

# TfidfVectorizer combines CountVectorizer and TfidTransformer with a single function.
tfidf = TfidfVectorizer(strip_accents=None, # Already preprocessed
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stops, None], # Not this time, but use idf with normalization
               'vect__tokenizer': [tokenizer],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stops, None], # Not this time
               'vect__tokenizer': [tokenizer],
               'vect__use_idf':[False],       # Raw counts without normalization 
               'vect__norm':[None],           # --------------||----------------
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
             {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stopsPorter, None], # Not this time, but use idf with normalization
               'vect__tokenizer': [tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stopsPorter, None], # Not this time
               'vect__tokenizer': [tokenizer_porter],
               'vect__use_idf':[False],       # Raw counts without normalization 
               'vect__norm':[None],           # --------------||----------------
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0, solver='saga'))])
# Solver specified to silence warning and to enable l1 regularization

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=1) # Number of jobs different from 1 sometimes crashes on Windows.

In [0]:
# The fitting of 2*2*2*3*5*2 models took around 30-60 minutes to fit in 2018. In 2019 it takes several hours. :(.
# Lowering the number of samples or parameters will make it quicker, but may reduce the performance greatly.
gs_lr_tfidf.fit(X_train, y_train)

In [0]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [0]:
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

# Load pikcle data and Implement NAIVE gaussian clf 

In [0]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

# Define basic tokenizer and Porter stemmer version
def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [0]:
import nltk

# Update to most resent stop-words
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [0]:
from nltk.corpus import stopwords

# Combine tokenizer with Porter stemmer and stop-word removal
stop = stopwords.words('english')
#[w for w in tokenizer_porter('a runner likes running and runs a lot')
#if w not in stop]

In [0]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movie_data.csv')
df.head()

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


In [0]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [0]:
stops = []
for s in stop:
    stops.append(tokenizer(s)[0])
stopsPorter = []
for s in stop:
    stopsPorter.append(tokenizer_porter(s)[0])

In [0]:
# To open an object that has been pickled, you need to import the object's dependencies and local functions
import pickle
with open('/content/drive/My Drive/Colab Notebooks/gs_lr_tfidf.pickle', 'rb') as f:
    gs_lr_tfidf = pickle.load(f)

In [0]:
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 1.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer_porter at 0x7fcdcad26950>} 
CV Accuracy: 0.876


#Use of NAIVE Gaussian by using best params from Logistic regression

Use these params :

Best parameter set: {'clf__C': 1.0, 
                    'clf__penalty': 'l1', 
                     'vect__ngram_range': (1, 1), 
                     'vect__stop_words': None, 
                     'vect__tokenizer': <function tokenizer_porter at 0x7efcd931e6a8>} 


In [0]:
# THIS IS SAMPLE CODE FOR REFERENCE 

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.ensemble import RandomForestClassifier



##------------------Below is original code from class teaching

# Pipeline :  In layman language , this is nothing but the sequence of transformation the data will go before putting to final estimator (eg. clf MultinomialNB)

# GridSearch is nothing but it will iterate with iterator param_grid over Pipeline 

# TfidfVectorizer combines CountVectorizer and TfidTransformer with a single function.
tfidf = TfidfVectorizer(strip_accents=None, # Already preprocessed
                        lowercase=False,
                        preprocessor=None)

param_grid = {'vect__ngram_range': [(1, 1)],
              'vect__stop_words': [stopsPorter, None], # Not this time, but use idf with normalization
              'vect__tokenizer': [tokenizer_porter],
              'clf__n_estimators': [4], 
              'clf__max_features': ['log2'],
              'clf__criterion': ['entropy'], 
              'clf__max_depth': [2],
              'clf__min_samples_split': [2],
              'clf__min_samples_leaf': [1]}

NB_tfidf = Pipeline([('vect', tfidf),
                     ('clf', RandomForestClassifier())])
# Solver specified to silence warning and to enable l1 regularization

gs_NB_tfidf = GridSearchCV(estimator=NB_tfidf, param_grid=param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=2,
                           n_jobs=1) # Number of jobs different from 1 sometimes crashes on Windows.


In [0]:
# The fitting of 2*2*2*3*5*2 models took around 30-60 minutes to fit in 2018. In 2019 it takes several hours. :(.
# Lowering the number of samples or parameters will make it quicker, but may reduce the performance greatly.
gs_NB_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits
[CV] clf__criterion=entropy, clf__max_depth=2, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=4, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'our', 'ourselv', 'you', "you'r", "you'v", "you'll", "you'd", 'your', 'your', 'yourself', 'yourselv', 'he', 'him', 'hi', 'himself', 'she', "she'", 'her', 'her', 'herself', 'it', "it'", 'it', 'itself', 'they', 'them', 'their', 'their', 'themselv', 'what', 'which', 'who', 'whom', 'thi', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'wa', 'were', 'be', 'been', 'be', 'have', 'ha', 'had', 'have', 'do', 'doe', 'did', 'do', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'becaus', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'dure', 'befor', 'after', 'abov', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again'

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf__criterion=entropy, clf__max_depth=2, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=4, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'our', 'ourselv', 'you', "you'r", "you'v", "you'll", "you'd", 'your', 'your', 'yourself', 'yourselv', 'he', 'him', 'hi', 'himself', 'she', "she'", 'her', 'her', 'herself', 'it', "it'", 'it', 'itself', 'they', 'them', 'their', 'their', 'themselv', 'what', 'which', 'who', 'whom', 'thi', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'wa', 'were', 'be', 'been', 'be', 'have', 'ha', 'had', 'have', 'do', 'doe', 'did', 'do', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'becaus', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'dure', 'befor', 'after', 'abov', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'onc', 'here', 'there', 'when', 'where

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.5min remaining:    0.0s
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf__criterion=entropy, clf__max_depth=2, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=4, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'our', 'ourselv', 'you', "you'r", "you'v", "you'll", "you'd", 'your', 'your', 'yourself', 'yourselv', 'he', 'him', 'hi', 'himself', 'she', "she'", 'her', 'her', 'herself', 'it', "it'", 'it', 'itself', 'they', 'them', 'their', 'their', 'themselv', 'what', 'which', 'who', 'whom', 'thi', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'wa', 'were', 'be', 'been', 'be', 'have', 'ha', 'had', 'have', 'do', 'doe', 'did', 'do', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'becaus', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'dure', 'befor', 'after', 'abov', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'onc', 'here', 'there', 'when', 'where

...

[CV]  clf__criterion=entropy, clf__max_depth=2, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=4, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'our', 'ourselv', 'you', "you'r", "you'v", "you'll", "you'd", 'your', 'your', 'yourself', 'yourselv', 'he', 'him', 'hi', 'himself', 'she', "she'", 'her', 'her', 'herself', 'it', "it'", 'it', 'itself', 'they', 'them', 'their', 'their', 'themselv', 'what', 'which', 'who', 'whom', 'thi', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'wa', 'were', 'be', 'been', 'be', 'have', 'ha', 'had', 'have', 'do', 'doe', 'did', 'do', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'becaus', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'dure', 'befor', 'after', 'abov', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'onc', 'here', 'there', 'when', 'where

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


[CV]  clf__criterion=entropy, clf__max_depth=2, clf__max_features=log2, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=4, vect__ngram_range=(1, 1), vect__stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'our', 'ourselv', 'you', "you'r", "you'v", "you'll", "you'd", 'your', 'your', 'yourself', 'yourselv', 'he', 'him', 'hi', 'himself', 'she', "she'", 'her', 'her', 'herself', 'it', "it'", 'it', 'itself', 'they', 'them', 'their', 'their', 'themselv', 'what', 'which', 'who', 'whom', 'thi', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'wa', 'were', 'be', 'been', 'be', 'have', 'ha', 'had', 'have', 'do', 'doe', 'did', 'do', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'becaus', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'dure', 'befor', 'after', 'abov', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'onc', 'here', 'there', 'when', 'where

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed: 14.9min finished
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=False,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                         

In [0]:
# Pickle (store to disk) the Grid Search CV object
import pickle
with open('gs_NB_tfidf.pickle', 'wb') as f:
    pickle.dump(gs_NB_tfidf, f, pickle.HIGHEST_PROTOCOL)

In [0]:
# To open an object that has been pickled, you need to import the object's dependencies and local functions
import pickle
with open('gs_NB_tfidf.pickle', 'rb') as f:
    gs_lr_tfidf = pickle.load(f)

In [0]:
print('Best parameter set: %s ' % gs_NB_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_NB_tfidf.best_score_)

Best parameter set: {'clf__criterion': 'entropy', 'clf__max_depth': 2, 'clf__max_features': 'log2', 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 4, 'vect__ngram_range': (1, 1), 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'our', 'ourselv', 'you', "you'r", "you'v", "you'll", "you'd", 'your', 'your', 'yourself', 'yourselv', 'he', 'him', 'hi', 'himself', 'she', "she'", 'her', 'her', 'herself', 'it', "it'", 'it', 'itself', 'they', 'them', 'their', 'their', 'themselv', 'what', 'which', 'who', 'whom', 'thi', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'wa', 'were', 'be', 'been', 'be', 'have', 'ha', 'had', 'have', 'do', 'doe', 'did', 'do', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'becaus', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'dure', 'befor', 'after', 'abov', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'th

In [0]:
clf = gs_NB_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.509


In [0]:
#from sklearn.metrics import accuracy_score

#score = accuracy_score(y_test, preds)
#print (score)

#Training of NAIVE Without using BEST params from logistic regression

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

#Train and evaluate the model
vect = TfidfVectorizer().fit(X_train)
X_train_vectorized = vect.transform(X_train)
clfrNB = MultinomialNB(alpha = 0.1)
clfrNB.fit(X_train_vectorized, y_train)
preds = clfrNB.predict(vect.transform(X_test))

In [0]:
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, preds)
print (score)

0.85668


In [0]:
# Pickle (store to disk) the Grid Search CV object
import pickle
with open('gs_lr_tfidf.pickle', 'wb') as f:
    pickle.dump(gs_lr_tfidf, f, pickle.HIGHEST_PROTOCOL)

# Use LDA for Unsupervised learning
Latent Dirichlet Allocation with scikit-learn

In [0]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [0]:


#Words that occur across too many documents are exluded
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,         # Words that occur across too many documents are exluded
                        max_features=5000) # Most frequent words, limiting the dimensionality
                                           # Both can be tuned
X = count.fit_transform(df['review'].values)

# This may take 5+ minutes to compute

from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10,
                                random_state=123,
                                learning_method='batch')

# 'batch' uses all data in one go (most accurate), but slower than 'online' (online/mini-batch)

X_topics = lda.fit_transform(X)
lda.components_.shape

#Results of LDA
#Print the 5 most important words for each of the 10 topics

n_top_words = 5
feature_names = count.get_feature_names()

for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx + 1))
    print(" ".join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
worst minutes awful script stupid
Topic 2:
family mother father children girl
Topic 3:
american war dvd music tv
Topic 4:
human audience cinema art sense
Topic 5:
police guy car dead murder
Topic 6:
horror house sex girl woman
Topic 7:
role performance comedy actor performances
Topic 8:
series episode war episodes tv
Topic 9:
book version original read novel
Topic 10:
action fight guy guys cool


In [0]:
horror = X_topics[:, 5].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:3]):
    print('\nHorror movie #%d:' % (iter_idx + 1))
    print(df['review'][movie_idx][:300], '...')


Horror movie #1:
House of Dracula works from the same basic premise as House of Frankenstein from the year before; namely that Universal's three most famous monsters; Dracula, Frankenstein's Monster and The Wolf Man are appearing in the movie together. Naturally, the film is rather messy therefore, but the fact that ...

Horror movie #2:
Okay, what the hell kind of TRASH have I been watching now? "The Witches' Mountain" has got to be one of the most incoherent and insane Spanish exploitation flicks ever and yet, at the same time, it's also strangely compelling. There's absolutely nothing that makes sense here and I even doubt there  ...

Horror movie #3:
<br /><br />Horror movie time, Japanese style. Uzumaki/Spiral was a total freakfest from start to finish. A fun freakfest at that, but at times it was a tad too reliant on kitsch rather than the horror. The story is difficult to summarize succinctly: a carefree, normal teenage girl starts coming fac ...
